# Monte Carlo Method
## Monte Carlo Integration
Monte Carlo integration is a technique for numerical integration using random numbers.

Consider a function $f(x):\Omega \to \mathbb{R}$ defined on $\Omega \subset \mathbb{R}^m$. We wish to calculate 
$$I:=\int_\Omega f(x)dx.$$
Let
$$V:=\int_\Omega 1dx$$
be the volume of $\Omega$, and
$$x_1,x_2,\cdots,x_n\in\Omega$$
are $n$ random points chosen uniformly in $\Omega$.

Then
$$I = \lim\limits_{n\to \infty}V\frac{1}{n}\sum\limits_{i=1}^{n}f(x_i).$$

Therefore, $I$ can be aprroximated by
$$I \approx V\frac{1}{n}\sum\limits_{i=1}^{n}f(x_i).$$

References : [Monte Carlo integration](https://en.wikipedia.org/wiki/Monte_Carlo_integration), [Monte Carlo method](https://en.wikipedia.org/wiki/Monte_Carlo_method)

## Example1: Calculating $\pi$

We can use Monte Carlo Method to calculate $\pi$. Or in other words, calculate the area of the unit circle.

Consider the function
$$    f(x)= 
\begin{cases}
    1,& \text{if } x^2+y^2<1\\
    0,              & \text{otherwise}
\end{cases}$$
and $\Omega = [0,1]\times[0,1]$ with $V=1$.

Consider $n$ random points $\{x_i\}_{i=1}^{n}$ in $\Omega$.
Then 
$$\frac{\pi}{4}=I \approx \frac{1}{n}\sum\limits_{i=1}^{n}f(x_i)$$

So 
$$\pi \approx 4\frac{1}{n}\sum\limits_{i=1}^{n}f(x_i)$$

We now apply this method with different $n$.

In [1]:
using LinearAlgebra

In [2]:
x1 = rand(2)

2-element Array{Float64,1}:
 0.9948098812484252
 0.6776122640870839

In [3]:
# Starts with n points
n=10^2;

# Define a variable to calculate the number of points that is inside the circle
pi_mc = 0;

# Run the loop for i=1...n
for ii=1:n
    x1 = rand(2);
    if(norm(x1,2)<1);
        pi_mc = pi_mc+1;
    end
end
println(4*pi_mc/n)

3.16


In [4]:
n=10^3;
pi_mc = 0;
for ii=1:n
    x1 = rand(2);
    if(norm(x1,2)<1);
        pi_mc = pi_mc+1;
    end
end
println(4*pi_mc/n)

3.168


In [5]:
n=10^4;
pi_mc = 0;
for ii=1:n
    x1 = rand(2);
    if(norm(x1,2)<1);
        pi_mc = pi_mc+1;
    end
end
println(4*pi_mc/n)

3.1452


In [6]:
n=10^5;
pi_mc = 0;
for ii=1:n
    x1 = rand(2);
    if(norm(x1,2)<1);
        pi_mc = pi_mc+1;
    end
end
println(4*pi_mc/n)

3.14384


In [7]:
n=10^6;
pi_mc = 0;
for ii=1:n
    x1 = rand(2);
    if(norm(x1,2)<1);
        pi_mc = pi_mc+1;
    end
end
println(4*pi_mc/n)

3.139612


In [8]:
n=10^7;
pi_mc = 0;
for ii=1:n
    x1 = rand(2);
    if(norm(x1,2)<1);
        pi_mc = pi_mc+1;
    end
end
println(4*pi_mc/n)

3.1421648


In [9]:
n=10^8;
pi_mc = 0;
for ii=1:n
    x1 = rand(2);
    if(norm(x1,2)<1);
        pi_mc = pi_mc+1;
    end
end
println(4*pi_mc/n)

3.14164036


In [10]:
n=10^4;
pi_mc = 0;
for nn=1:n
    for ii=1:10
        x1 = rand(2);
        x1[1] = (x1[1] + (ii-1))*0.1;
        if(norm(x1,2)<1);
            pi_mc = pi_mc+1;
        end
    end
end
println(4*pi_mc/(10*n))

3.14628


## Example 2:

In [11]:
rand(2)

2-element Array{Float64,1}:
 0.5765529980890403 
 0.24028097696677375

In [12]:
n=100;
pi_mc = 0;
for nn=1:n
for ii=1:10
    x1 = rand(2);
    for jj=1:10
        x1[1] = (x1[1] + (ii-1))*0.1;
        x1[2] = (x1[2] + (jj-1))*0.1;
        if(norm(x1,2)<1);
            pi_mc = pi_mc+1;
        end
    end
end
end
println(4*pi_mc/(100*n))

2.9764


In [13]:
n=1000;
pi_mc = 0;
for nn=1:n
for ii=1:10
    x1 = rand(2);
    for jj=1:10
        x1[1] = (x1[1] + (ii-1))*0.1;
        x1[2] = (x1[2] + (jj-1))*0.1;
        if(norm(x1,2)<1);
            pi_mc = pi_mc+1;
        end
    end
end
end
println(4*pi_mc/(100*n))

2.9782


In [14]:
n=1000000;
pi_mc = 0;
for nn=1:n
for ii=1:10
    x1 = rand(2);
    for jj=1:10
        x1[1] = (x1[1] + (ii-1))*0.1;
        x1[2] = (x1[2] + (jj-1))*0.1;
        if(norm(x1,2)<1);
            pi_mc = pi_mc+1;
        end
    end
end
end
println(4*pi_mc/(100*n))

2.97719824
